In [1]:
#from time import time
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pandas as pd

from my_fun import read_brain_feature
from my_fun import run_model
from my_fun import summary_result
from my_fun import plot_result_mergecv

print(sklearn.__version__)
#brain_feature_list, subject_id = read_brain_feature()
brain_feature_list, subject_id = read_brain_feature_fc()

# read subject_info:
pnc_subject_info = pd.read_csv("data/pnc_subject_info.csv", delimiter = ",")
print(pnc_subject_info.shape)

subject_info_merge = subject_id.merge(pnc_subject_info, how = "left", left_on = "names", right_on = "SUBJID")
print(subject_info_merge.shape)

# head_motion_subjects = pd.read_csv("data/head_motion_subject_delete.txt", delimiter = "\t", header = None)
# #print(head_motion_subjects)
# head_motion_sublist = head_motion_subjects.iloc[:,0].tolist()
# subject_info_merge = subject_info_merge[~subject_info_merge.SUBJID.isin(head_motion_sublist)]

# remove samples with Med_rating higher than 1:
subject_info_merge["Med_Rating"] = subject_info_merge["Med_Rating"].apply(pd.to_numeric, errors='coerce')
subject_info_merge = subject_info_merge.loc[subject_info_merge["Med_Rating"]<2,:]

subject_info_merge['Sex'] = subject_info_merge['Sex'].apply({'F':0, 'M':1}.get)
      
print(subject_info_merge.head())
subject_info_columns = ['SUBJID', 'Sex', 'age_at_cnb']


0.19.1


NameError: name 'read_brain_feature_fc' is not defined

In [ ]:
print("participants:")
print("number of subjects: %d" % subject_info_merge.shape[0])
print("meam  and std of age: %.2f %.2f" % (subject_info_merge["age_at_cnb"].mean(),\
                                           subject_info_merge["age_at_cnb"].std()))
print("range of age: %d - %d" % (subject_info_merge["age_at_cnb"].max(),\
                                 subject_info_merge["age_at_cnb"].min()))

print("number of females: %d" % subject_info_merge.loc[subject_info_merge['Sex']==0,:].shape[0])
print("number of males: %d" % subject_info_merge.loc[subject_info_merge['Sex']==1,:].shape[0])



In [ ]:
from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

subject_info_hc = subject_info_merge

# tune model in 3-fold CV:
# n fold CV on hc data:
nfold = 10
seed = 111
kf = KFold(n_splits=nfold, shuffle = True, random_state=seed)
# kf = StratifiedKFold(n_splits=nfold, shuffle = True, random_state=seed)


In [ ]:
###############################################ridge:#########################################
from sklearn.linear_model import RidgeCV
import numpy as np

# ridge regression:
alpha = np.power(10, np.linspace(-3, 3, num = 10))
# print(alpha)
ridge_cv = RidgeCV(alphas=alpha, cv = nfold)

result_table_ridge, scatter_data_ridge = run_model(ridge_cv, subject_info_hc, brain_feature_list, 
                                                   kf, fit_method = 0)


In [ ]:
result_table_ridge.to_csv("result/result_table_ridge.csv", sep = ",")
scatter_data_ridge.to_csv("result/scatter_data_ridge.csv", sep = ",", index = False)


In [ ]:
import pandas as pd
result_table = pd.read_csv("result/result_table_ridge.csv", sep = ",", header = [0,1], index_col = 0)
# result_table.columns = col_result_table
print(result_table)
result_summary_ridge, result_plot_ridge = summary_result(result_table)

In [ ]:
# plot ridge regression results:

scatter_data = pd.read_csv("result/scatter_data_ridge.csv", sep = ",", header = 0, )
plot_result_mergecv(result_summary_ridge, scatter_data)

In [ ]:
############################################### SVR: #########################################
from sklearn.svm import SVR
import numpy as np

C = np.power(10, np.linspace(-1,2, num = 20))
epsilon=np.linspace(.001, 1, num = 20)

param_grid_svr = dict(C = C, epsilon = epsilon)
# linear model takes very long time to run.
#grid_svr = GridSearchCV(estimator=SVR(kernel='linear'), param_grid=param_grid_svr, n_jobs=3, refit = True)
# use default kernel rbf:
grid_svr = GridSearchCV(estimator=SVR(), param_grid=param_grid_svr, n_jobs=3, refit = True)

result_table_svr, scatter_data_svr = run_model(grid_svr, subject_info_hc, brain_feature_list, kf, fit_method = 1)



In [ ]:
result_table_svr.to_csv("result/result_table_svr.csv", sep = ",")
scatter_data_svr.to_csv("result/scatter_data_svr.csv", sep = ",", index = False)

In [ ]:
import pandas as pd

result_table = pd.read_csv("result/result_table_svr.csv", sep = ",", header = [0,1], index_col = 0)
result_summary_svr, result_plot_svr = summary_result(result_table)

#print(plot_data)

In [ ]:
# plot svr regression results:

scatter_data = pd.read_csv("result/scatter_data_svr.csv", sep = ",", header = 0, )
plot_result_mergecv(result_summary_svr, scatter_data)

In [ ]:
############################################### GPR: #########################################
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel as CK

# gpr_kernel = 1.0 * RBF(length_scale=10, length_scale_bounds=(1e-2, 1e3)) \
#                     + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# gpr_kernel2 = 1.0 * RBF(length_scale=100, length_scale_bounds=(1e-2, 1e3)) \
#                     + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# gpr_kernel3 = 1.0 * RBF(length_scale=1000, length_scale_bounds=(1e-2, 1e3)) \
#                     + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

gpr_kernel4 = CK(2.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) \
                    + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))


# gpr_kernel4 = 1.0 * RBF(length_scale=100, length_scale_bounds=(1e-2, 1e3))

gpr_alpha = np.power(10, np.linspace(-2, 2, num = 10))

#gpr_alpha = np.linspace(0,.1, num = 3)

#param_grid_gpr = dict(alpha = gpr_alpha, kernel=[gpr_kernel, gpr_kernel2, gpr_kernel3, gpr_kernel4])
param_grid_gpr = dict(alpha = gpr_alpha, kernel=[gpr_kernel4])

grid_gpr = GridSearchCV(estimator=GaussianProcessRegressor(), \
                                param_grid=param_grid_gpr, n_jobs=3, refit = True)

result_table_gpr, scatter_data_gpr = run_model(grid_gpr, subject_info_hc, brain_feature_list, kf, fit_method = 1)

In [ ]:
result_table_gpr.to_csv("result/result_table_gpr.csv", sep = ",")
scatter_data_gpr.to_csv("result/scatter_data_gpr.csv", sep = ",", index = False)

In [ ]:
import pandas as pd

result_table = pd.read_csv("result/result_table_gpr.csv", sep = ",", header = [0,1], index_col = 0)
result_summary_gpr, result_plot_gpr = summary_result(result_table)

#print(plot_data)

In [ ]:
# plot gpr regression results:
from my_fun import plot_result_mergecv

# to make the plot show up if codes are called in a function.
%matplotlib inline

scatter_data = pd.read_csv("result/scatter_data_gpr.csv", sep = ",", header = 0, )
plot_result_mergecv(result_summary_gpr, scatter_data)
        